
# Open Power System (time series) data analysis

https://open-power-system-data.org/ for daily consumption, wind, solar, solar+wind data.

Credit: https://www.dataquest.io/blog/tutorial-time-series-analysis-with-pandas/

Data: https://github.com/jenfly/opsd/raw/master/opsd_germany_daily.csv

We will use:
* pandas
* matplotlib
* seaborn

In [ ]:
import pandas as pd

## Date objects

Datetime objects are timezone aware or unaware, generally you can't combine the two as it makes no sense to compare with and without a timezone. 

Generally speaking you're better off if you can convert a timeseries into one with a timezone (if you know it), then you preserve the timezone information. 

In this example we don't have timezones so we're working with timezone unaware dates.

In [ ]:
pd.to_datetime('2018-01-15 3:45pm')

In [ ]:
pd.to_datetime('2018-01-15') 

In [ ]:
pd.to_datetime('2018-01-15T15:45:00Z') # ISO 8601 is a sane way to consistently deal with dates!

In [ ]:
pd.to_datetime('2018-01-15 3:45pm', utc=True) # note inclusion of UTC

In [ ]:
pd.to_datetime('7/8/1952') # defaults to MM/DD/YY

In [ ]:
pd.to_datetime('7/8/1952', dayfirst=True) # you can override for DD/MM/YY

In [ ]:
# we can generate ranges easily
pd.date_range(start="2018-01-01", freq="1D", periods=10)

Frequencies look like:
* D daily, B business daily (e.g. 5D means 5 day gaps)
* W weekly, M monthly, Q quarterly, A annually
* H hourly, min by the minute

See them here: https://pandas.pydata.org/pandas-docs/stable/user_guide/timeseries.html#dateoffset-objects

We might use ranges to:

* clean up imported data that lacks datetime information
* create a mask to select ranges of data from a larger set

In [ ]:
pd.timedelta_range(start='1 day', end='5 day', periods=5)

In [ ]:
pd.timedelta_range(start='1 day', periods=24, freq='H')

## Loading our data

Convenience functiosn include `read_csv`, `read_table`, `read_hdf5`, `read_sql` and more.

In [ ]:
opsd_daily = pd.read_csv('opsd_germany_daily.csv')
opsd_daily.shape

In [ ]:
opsd_daily.head()

In [ ]:
opsd_daily.tail(3)

In [ ]:
opsd_daily.dtypes

In [ ]:
opsd_daily = opsd_daily.set_index('Date')
opsd_daily.head(3)

In [ ]:
opsd_daily.index

In [ ]:
# this is a shortcut method to load by setting the index to a known column
opsd_daily = pd.read_csv('opsd_germany_daily.csv', index_col=0, parse_dates=True)

In [ ]:
opsd_daily.head()

In [ ]:
# Add columns with year, month, and weekday name
opsd_daily['Year'] = opsd_daily.index.year
opsd_daily['Month'] = opsd_daily.index.month
opsd_daily['Weekday Name'] = opsd_daily.index.day_name()
opsd_daily['Weekday'] = opsd_daily.index.weekday
# isin lets us test for set membership
opsd_daily['Is Weekend'] = opsd_daily['Weekday'].isin((5, 6))
opsd_daily.head()

In [ ]:
opsd_copy = opsd_daily.reset_index()
opsd_copy.head()

In [ ]:
#opsd_copy['Date'].year # won't work
opsd_copy['Date'].dt.year.sample(5) # will work - off of an index we have a regular column
# we need to dip down one level to get the datetime tools

## Reviewing the data

In [ ]:
# Display a random sampling of 5 rows
opsd_daily.sample(5, random_state=0)

In [ ]:
opsd_daily['Weekday Name'].value_counts()

In [ ]:
opsd_daily.describe()

## Indexing

In [ ]:
opsd_daily.loc['2017-08-10']

In [ ]:
# note INCLUSIVE end indexing - different to usual indexing!
# THIS IS NOVEL - BE AWARE!
opsd_daily.loc['2014-01-20':'2014-01-22']

In [ ]:
opsd_daily.loc['2012-02'] # partial matches are supported

## Visualising

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline
# Display figures inline in Jupyter notebook

In [ ]:
import seaborn as sns # Seaborn is a statistical plotting library
# Use seaborn style defaults and set the default figure size
sns.set(rc={'figure.figsize':(11, 4)})

In [ ]:
opsd_daily['Consumption'].plot(linewidth=0.5);

In [ ]:
cols_plot = ['Consumption', 'Solar', 'Wind']
axes = opsd_daily[cols_plot].plot(marker='.', alpha=0.5, linestyle='None', figsize=(11, 9), subplots=True)
for ax in axes:
    ax.set_ylabel('Daily Totals (GWh)')

We see two general behaviours for Consumption (hypothesis - weekday and weekend?). 

Solar starts later than Wind with many years of missing (NaN) data.

Consumption is higher in winter. Wind and Solar production appear to be increasing over time.

### What patterns do you see below?

In [ ]:
ax = opsd_daily.loc['2017', 'Consumption'].plot()
ax.set_ylabel('Daily Consumption (GWh)');
ax.set_title("2017's Consumption by day");

### Exercise - can you group the data to get monthly means for Consumption?

You'll first want to get the 2017 subset of data for `['Consumption', 'Month']`. Does it look sensible?

Next you can `groupby` on the Month column. Next you need to call one of the functions, we're after the mean but there's also sum, median, std, var, min etc. Try these

In [ ]:
# your code here

In [ ]:
# now take the above answer and add a .plot() to graph it
# plot returns an axis object, if you take `ax = <your code>` then on a new line you can add a title like
# ax.set_title('Mean monthly consumption for 2017');

### Drilling further - it looks like we do have daily behaviour

In [ ]:
ax = opsd_daily.loc['2017-01':'2017-02', 'Consumption'].plot(marker='o', linestyle='-')
ax.set_ylabel('Daily Consumption (GWh)');

In [ ]:
import matplotlib.dates as mdates

In [ ]:
fig, ax = plt.subplots()
ax.plot(opsd_daily.loc['2017-01':'2017-02', 'Consumption'], marker='o', linestyle='-')
ax.set_ylabel('Daily Consumption (GWh)')
ax.set_title('Jan-Feb 2017 Electricity Consumption')
# Set x-axis major ticks to weekly interval, on Mondays
ax.xaxis.set_major_locator(mdates.WeekdayLocator(byweekday=mdates.MONDAY))
# Format x-tick labels as 3-letter month name and day number
ax.xaxis.set_major_formatter(mdates.DateFormatter('%b %d'));

### Grouping on the weekday to see mean behaviour by day of week

In [ ]:
mean_by_weekday = opsd_daily.loc['2017', ['Consumption', 'Weekday']].groupby('Weekday').mean()
mean_by_weekday

In [ ]:
mean_by_weekday.plot();

In [ ]:
day_of_week = {0:'Monday', 1:'Tuesday', 2:'Wednesday', 3:'Thursday', 4:'Friday', 5:'Saturday', 6:'Sunday'}
new_index = mean_by_weekday.index.map(day_of_week)
mean_by_weekday.set_index(new_index)

In [ ]:
mean_by_weekday.set_index(new_index).plot();

## Summarising seasonality with seaborn

What patterns do we see below? General patterns? Outliers?

This is for all data, not just 2017, so what might we conclude?

In [ ]:
fig, axes = plt.subplots(3, 1, figsize=(11, 10), sharex=True)
for name, ax in zip(['Consumption', 'Solar', 'Wind'], axes):
    sns.boxplot(data=opsd_daily, x='Month', y=name, ax=ax)
    ax.set_ylabel('GWh')
    ax.set_title(name)
    # Remove the automatic x-axis label from all but the bottom subplot
    if ax != axes[-1]:
        ax.set_xlabel('')

In [ ]:
sns.boxplot(data=opsd_daily, x='Weekday Name', y='Consumption');
# note ordering is built from the natural order in the underlying data

In [ ]:
day_of_week

In [ ]:
day_of_week.values()

In [ ]:
# What are the outliers telling us? 
sns.boxplot(data=opsd_daily, x='Weekday Name', y='Consumption', order=day_of_week.values());

In [ ]:
# are any of these German holidays? I'm curious!
# New Year's Eve, Christmas Eve
daily_mask = opsd_daily['Weekday Name'] == 'Monday'
opsd_daily[daily_mask].query("Consumption < 1000").sort_values('Month')

## Autocorrelation and lags

How similar is today's point to the same point N days in the future? Autocorrelation tests all frequencies.

Lags look at 1 frequency (default is 1 unit ahead, for us that's 1 day).

In [ ]:
from pandas.plotting import autocorrelation_plot

In [ ]:
autocorrelation_plot(opsd_daily.Consumption)

In [ ]:
fig, ax = plt.subplots();
autocorrelation_plot(opsd_daily.Consumption, ax=ax)
ax.set_xlim(0, 30);

In [ ]:
fig, ax = plt.subplots();
autocorrelation_plot(opsd_daily.Consumption, ax=ax)
ax.set_xlim(0, 360);

Lag plot shows structure between $y(t)$ and $y(t+1)$. A visual relationships suggest that there's structure in the data.

In [ ]:
from pandas.plotting import lag_plot

data = opsd_daily.loc['2013']
lag_plot(data['Consumption']);

In [ ]:
lag_plot(data['Consumption'], c=data['Is Weekend'][:-1], cmap='viridis');

In [ ]:
# check the days to day-names list
opsd_daily[['Weekday', 'Weekday Name']].drop_duplicates().sort_values('Weekday')

### Exercise - can you show the lag plot by day of week using Weekday?

Viridis goes from purple (0) to green to yellow (1) - so do we see daily structure?

In [ ]:
# your code here

## Date ranges again and resampling

In [ ]:
# note that freq is set
pd.date_range('2004-09-20', periods=8, freq='H')

In [ ]:
opsd_daily.index[:5] # note that freq isn't as it came from a file

In [ ]:
# let's make a small copy
times_sample = pd.to_datetime(['2013-02-03', '2013-02-06', '2013-02-08'])
# Select the specified dates and just the Consumption column
consum_sample = opsd_daily.loc[times_sample, ['Consumption']].copy()
consum_sample

In [ ]:
# Convert the data to daily frequency, without filling any missings
consum_freq = consum_sample.asfreq('D')
consum_freq

In [ ]:
# Create a column with missings forward filled
consum_freq['Consumption - Forward Fill'] = consum_sample.asfreq('D', method='ffill')
consum_freq

### Weekly resampling - downsampling our data

In [ ]:
# Specify the data columns we want to include (i.e. exclude Year, Month, Weekday Name)
data_columns = ['Consumption', 'Wind', 'Solar', 'Wind+Solar']
# Resample to weekly frequency, aggregating with mean
opsd_weekly_mean = opsd_daily[data_columns].resample('W').mean()
opsd_weekly_mean.tail(3)

In [ ]:
# What's going on in this code? What do we expect to see?

# Start and end of the date range to extract
start, end = '2017-01', '2017-06'
# Plot daily and weekly resampled time series together
fig, ax = plt.subplots()
ax.plot(opsd_daily.loc[start:end, 'Solar'],
marker='.', linestyle='-', linewidth=0.5, label='Daily')
ax.plot(opsd_weekly_mean.loc[start:end, 'Solar'],
marker='o', markersize=8, linestyle='-', label='Weekly Mean Resample')
ax.set_ylabel('Solar Production (GWh)')
ax.legend();

## Rolling means

In [ ]:
# Compute the centered 7-day rolling mean (not centrered, using history only)
opsd_7d = opsd_daily[data_columns].rolling(7, center=False).mean()
opsd_7d.head(10)

In [ ]:
# Start and end of the date range to extract
start, end = '2017-01', '2017-06'
# Plot daily, weekly resampled, and 7-day rolling mean time series together
fig, ax = plt.subplots()
ax.plot(opsd_daily.loc[start:end, 'Solar'],
marker='.', linestyle='-', linewidth=0.5, label='Daily')
ax.plot(opsd_weekly_mean.loc[start:end, 'Solar'],
marker='o', markersize=8, linestyle='-', label='Weekly Mean Resample')
ax.plot(opsd_7d.loc[start:end, 'Solar'],
marker='.', linestyle='-', label='7-d Rolling Mean')
ax.set_ylabel('Solar Production (GWh)')
ax.legend();

# note that the 7d rolling is on a daily basis still
# the weekly mean is at a week-at-a-time granularity

## Trends

If we plot a 365 trend vs a 7 day trend, what can we see for overall consumption and green energy production?

In [ ]:
# The min_periods=360 argument accounts for a few isolated missing days in the
# wind and solar production time series
opsd_365d = opsd_daily[data_columns].rolling(window=365, center=False, min_periods=360).mean()

In [ ]:
# What can we observe in this plot?

# Plot daily, 7-day rolling mean, and 365-day rolling mean time series
fig, ax = plt.subplots(figsize=(14,4))
ax.plot(opsd_daily['Consumption'], marker='.', markersize=2, color='0.6',
linestyle='None', label='Daily')
ax.plot(opsd_7d['Consumption'], linewidth=2, label='7-d Rolling Mean')
ax.plot(opsd_365d['Consumption'], color='0.2', linewidth=3,
label='Trend (365-d Rolling Mean)')
# Set x-ticks to yearly interval and add legend and labels
ax.xaxis.set_major_locator(mdates.YearLocator())
ax.legend()
ax.set_xlabel('Year')
ax.set_ylabel('Consumption (GWh)')
ax.set_title('Trends in Electricity Consumption');

In [ ]:
# let's do the same for solar and wind

# Plot 365-day rolling mean time series of wind and solar power
fig, ax = plt.subplots()
for nm in ['Wind', 'Solar', 'Wind+Solar']:
    ax.plot(opsd_365d[nm], label=nm)
    # Set x-ticks to yearly interval, adjust y-axis limits, add legend and labels
    ax.xaxis.set_major_locator(mdates.YearLocator())
    ax.set_ylim(0, 400)
    ax.legend()
    ax.set_ylabel('Production (GWh)')
    ax.set_title('Trends in Electricity Production (365-d Rolling Means)');

## Share of green power over the years?

In [ ]:
# Compute the annual sums, setting the value to NaN for any year which has
# fewer than 360 days of data
opsd_annual = opsd_daily[data_columns].resample('A').sum(min_count=360)
# The default index of the resampled DataFrame is the last day of each year,
# ('2006-12-31', '2007-12-31', etc.) so to make life easier, set the index
# to the year component
opsd_annual = opsd_annual.set_index(opsd_annual.index.year)
opsd_annual.index.name = 'Year'
opsd_annual

### Exercise - compute the fraction of  wind+solar to overall Consumption and plot it

Add a y label, title and perhaps set the ylim to (0, 0.3)

In [ ]:
# your code here

# Conclusion

We've covered datetimes, resampling, various plotting approaches to reveal structure in our data and you've grouped your data and created some additional visualisations.

Now you might want to check:

* Prophet (Facebook) for time series modeling
* Timezones
* `statsmodels` time series decomposition